# Simple Ensemble of XGBoost and CatBoost

Combine predictions from both models using weighted averaging.

Weights: 0.5 for each model (equal weighting)

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_log_error
import warnings
warnings.filterwarnings('ignore')

print("Loading OOF predictions and creating ensemble...")

# Load training data for target
train_df = pd.read_csv('/home/code/data/train.csv')
y = train_df['Calories']

# Load OOF predictions from both models
oof_xgb = pd.read_csv('/home/code/experiments/oof_001_baseline_xgboost.csv')
oof_cat = pd.read_csv('/home/code/experiments/oof_002_catboost_baseline.csv')

print(f"XGB OOF shape: {oof_xgb.shape}")
print(f"CatBoost OOF shape: {oof_cat.shape}")
print(f"Target shape: {y.shape}")

# Check if IDs match
assert list(oof_xgb['id']) == list(oof_cat['id']) == list(train_df['id']), "ID mismatch!"

# Extract predictions
pred_xgb = oof_xgb['Calories'].values
pred_cat = oof_cat['Calories'].values

# Simple average ensemble
ensemble_pred = (pred_xgb + pred_cat) / 2.0

# Calculate CV score
ensemble_pred_clipped = np.clip(ensemble_pred, 0, None)
cv_score = np.sqrt(mean_squared_log_error(y, ensemble_pred_clipped))

print(f"\n{'='*50}")
print(f"XGBoost CV RMSLE: 0.209215")
print(f"CatBoost CV RMSLE: 0.202310")
print(f"Ensemble CV RMSLE: {cv_score:.6f}")
print(f"{'='*50}")

# Calculate individual fold scores if available (approximate)
print(f"\nEnsemble improvement: {(0.209215 - cv_score):.6f} vs XGBoost")
print(f"Ensemble improvement: {(0.202310 - cv_score):.6f} vs CatBoost")

Loading OOF predictions and creating ensemble...
XGB OOF shape: (8000, 2)
CatBoost OOF shape: (8000, 2)
Target shape: (8000,)

XGBoost CV RMSLE: 0.209215
CatBoost CV RMSLE: 0.202310
Ensemble CV RMSLE: 0.203915

Ensemble improvement: 0.005300 vs XGBoost
Ensemble improvement: -0.001605 vs CatBoost


In [2]:
# Load test predictions from both models
test_xgb = pd.read_csv('/home/code/submission_candidates/candidate_001_baseline_xgboost.csv')
test_cat = pd.read_csv('/home/code/submission_candidates/candidate_002_catboost_baseline.csv')

print(f"XGB test shape: {test_xgb.shape}")
print(f"CatBoost test shape: {test_cat.shape}")

# Check if IDs match
assert list(test_xgb['id']) == list(test_cat['id']), "ID mismatch in test!"

# Create ensemble prediction
ensemble_test = (test_xgb['Calories'].values + test_cat['Calories'].values) / 2.0

# Create submission
submission = pd.DataFrame({
    'id': test_xgb['id'],
    'Calories': ensemble_test
})

# Clip predictions to training data range
train_min = train_df['Calories'].min()
train_max = train_df['Calories'].max()
submission['Calories'] = submission['Calories'].clip(train_min, train_max)

print(f"\nSubmission shape: {submission.shape}")
print(f"Calories range: [{submission['Calories'].min():.2f}, {submission['Calories'].max():.2f}]")

# Save submission
submission_path = '/home/code/submission_candidates/candidate_003_ensemble_simple.csv'
submission.to_csv(submission_path, index=False)
print(f"Submission saved to: {submission_path}")

# Save OOF predictions for potential stacking
oof_df = pd.DataFrame({
    'id': train_df['id'],
    'Calories': ensemble_pred
})
oof_path = '/home/code/experiments/oof_003_ensemble_simple.csv'
oof_df.to_csv(oof_path, index=False)
print(f"OOF predictions saved to: {oof_path}")

XGB test shape: (2000, 2)
CatBoost test shape: (2000, 2)

Submission shape: (2000, 2)
Calories range: [57.55, 474.55]
Submission saved to: /home/code/submission_candidates/candidate_003_ensemble_simple.csv
OOF predictions saved to: /home/code/experiments/oof_003_ensemble_simple.csv
